# Examples

## Private LLM calls with santized prompts

In [12]:
import requests, json
api_url = "http://localhost:8000"

request = {
    "provider": "openai",
    "model": "gpt-3.5-turbo",
    "key_cache_prompt": "Rank the importance of these features of jobs for Rushabh and Jonah: 1. Compensation 2. Work Life Balance 3. Company Mission",
    "messages": [
        {
            "role": "system",
            "content": "Context: Rushabh is very passionate about technology and loves tinkering with new products. He enjoys spending quality time in nature in his free time. He's early in his career, so he's willing to forgo salary for career growth. Jonah is an entrepreneur at heart and believes that tech companies are the best suited for making lots of money. He's willing to work long hours, as long as he's working on a cause he believes in.",
            "name": "context"
        },
        {
            "role": "system",
            "content": "Please provide a JSON response with the following format: {name: [ordered list of features]}"
        },
        {
            "role": "user",
            "content": "Rank the importance of these features of jobs for Rushabh and Jonah: 1. Compensation 2. Work Life Balance 3. Company Mission",
            "name": "rankings"
        }
    ]
}

res = requests.post(api_url + "/llm/chat/completions?debug=true", json=request)
response = res.json()



####  What AI Sees

In [13]:
print(response)

{'messages': [{'role': 'assistant', 'content': '{"Rushabh": ["Company Mission", "Work Life Balance", "Compensation"], "Jonah": ["Compensation", "Work Life Balance", "Company Mission"]}', 'name': None, 'function': None}], 'provider_response': {'id': 'chatcmpl-81nanRFLeNmwMHLAbKakrf1PfbB8W', 'object': 'chat.completion', 'created': 1695439653, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"Rushabh": ["Company Mission", "Work Life Balance", "Compensation"], "Jonah": ["Compensation", "Work Life Balance", "Company Mission"]}', 'name': None}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 350, 'completion_tokens': 95, 'total_tokens': 445}}, 'raw_provider_response': None, 'raw_request': None}


In [10]:
print(json.dumps(response["raw_request"], indent=4))

null


####  What users see

In [11]:
print(json.dumps(response["messages"], indent=4))

[
    {
        "role": "assistant",
        "content": "{\"Rushabh\": [\"Company Mission\", \"Work Life Balance\", \"Compensation\"], \"Jonah\": [\"Compensation\", \"Work Life Balance\", \"Company Mission\"]}",
        "name": null,
        "function": null
    }
]


## Secure RAG with Santized Documents and Prompts



### Set Up Vector DB
We assume you are using postgres with PGVector. Please email sumant@pontus.so for any support for other vector databases.

In [110]:
documents_from_wiki = ["Bill Burr", "Marie Kondo", "Garry Kasparov"]
requests.post(api_url + "/demo/load_wiki_pages", json={"page_names": documents_from_wiki})

### Call LLM
To see what prompts are sent to OpenAI, please pass the query param debug as true (?debug=true) and to enable RAG (?rag=true)

In [111]:
request = {
	"provider": "openai",
	"model": "gpt-3.5-turbo",
	"messages": [

		{
			"role": "system",
			"content": "Please provide a JSON response with the following format: {'Cards against Humanity': [ordered list of Bill, Garry, or Marie], 'Go': [ordered list of Bill, Garry, or Marie], 'Solitare': [ordered list of Bill, Garry, or Marie]}"
		},
		{
			"role": "user",
			"content": "For each of the games 'Cards against Humanity', 'Go', and 'Solitaire', please order Marie Kondo, Garry Kasparov, and Bill Burr based on how much they would enjoy playing each game."
		}
	],
	"titles": ["Bill Burr", "Marie Kondo", "Garry Kasparov"]
}


res_with_rag = requests.post(api_url + "/llm/chat/completions?debug=true&enable_rag=true", json=request)
response_with_rag = res_with_rag.json()


#### What AI sees

In [112]:
print(json.dumps(response_with_rag["raw_request"], indent=4))

[
    {
        "role": "system",
        "content": "Ensure that all placeholders, including those inside quotes, are enclosed bythe greek letter alpha (\u03b1), exactly as I have done in this prompt. You MUST use the greek letter (\u03b1) to indicate placeholders. Use the context below to help answer the question and follow the desired format accurately.",
        "name": null,
        "function": null
    },
    {
        "role": "system",
        "content": "Please provide a JSON response with the following format: {'Cards against Humanity': [ordered list of \u03b1Zxe3Isuo5+b/KeCAf7e9V8R/cKyQtF/JBTaxnbfLo8E=\u03b1, \u03b1QXH3QWS3gZ6rk2EsubMlJaJ/Ktbz50Mr6XmfOn+O3yg=\u03b1, or \u03b1l32XxxE1aF6OefMnccK/k8IF4aqpXbBvEI/fNY4Unpo=\u03b1], 'Go': [ordered list of \u03b1Zxe3Isuo5+b/KeCAf7e9V8R/cKyQtF/JBTaxnbfLo8E=\u03b1, \u03b1QXH3QWS3gZ6rk2EsubMlJaJ/Ktbz50Mr6XmfOn+O3yg=\u03b1, or \u03b1l32XxxE1aF6OefMnccK/k8IF4aqpXbBvEI/fNY4Unpo=\u03b1], 'Solitare': [ordered list of \u03b1Zxe3Isuo5+b/KeCAf

#### What User See

In [113]:
print(json.dumps(response_with_rag["messages"], indent=4))

[
    {
        "role": "assistant",
        "content": "{'Cards against Humanity': [Bill Burr, Marie Kondo, Garry Kasparov], 'Go': [Marie Kondo, Garry Kasparov, Bill Burr], 'Solitaire': [Garry Kasparov, Marie Kondo, Bill Burr]}",
        "name": null,
        "function": null
    }
]
